# Colab Gradio Inference

Download model weights and helper script from your hosting location, then launch an interactive Gradio app that tiles the image,
 runs YOLO inference on GPU (when available), fuses detections with the graph-cut strategy, and visualises the final collembola
count.

In [ ]:
# Install minimal runtime dependencies. Comment out packages you already have in your runtime.
!pip install -q ultralytics gradio shapely pillow torch matplotlib requests

In [ ]:
import importlib
import requests
from pathlib import Path
import torch

# ---------------------------------------------------------------------------
# Configure your download locations here
# ---------------------------------------------------------------------------
REMOTE_BASE = 'https://collembot.ch/colab-resources/'  # hosting root with model, helper, and example image
MODEL_URL = REMOTE_BASE + 'collembot-2025_12-yolo11x-seg.pt'
HELPER_URL = REMOTE_BASE + 'gradio_app.py'
EXAMPLE_URL = REMOTE_BASE + 'example.jpg'

ROOT = Path('/content')
ROOT.mkdir(exist_ok=True)
weights_path = ROOT / 'collembot-2025_12-yolo11x-seg.pt'
helper_path = ROOT / 'gradio_app.py'
example_path = ROOT / 'example.jpg'

USE_GPU = torch.cuda.is_available()
PREFERRED_DEVICE = 'cuda:0' if USE_GPU else 'cpu'
USE_HALF = False
BATCH_SIZE = 12 if USE_GPU else 4

if USE_GPU:
    name = torch.cuda.get_device_name(torch.cuda.current_device())
    print(f'✅ GPU detected: {name}')
else:
    print('⚠️ No GPU detected. In Colab, choose GPU via Runtime > Change runtime type for faster inference.')
print(f'Using device={PREFERRED_DEVICE}, half={USE_HALF}, batch={BATCH_SIZE}')


def download(url: str, dest: Path):
    response = requests.get(url, stream=True)
    response.raise_for_status()
    dest.parent.mkdir(parents=True, exist_ok=True)
    with open(dest, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            if chunk:
                f.write(chunk)
    print(f'Downloaded {url} -> {dest}')


# Download the helper, weights, and example image. They will overwrite existing copies
# to keep the notebook reproducible.
download(HELPER_URL, helper_path)
download(MODEL_URL, weights_path)
download(EXAMPLE_URL, example_path)

# Import the freshly downloaded helper module
spec = importlib.util.spec_from_file_location('gradio_app', helper_path)
gradio_app = importlib.util.module_from_spec(spec)
spec.loader.exec_module(gradio_app)


In [ ]:
# Launch the Gradio UI. Sharing is disabled; run locally.
gradio_app.launch_app(
    model_path=str(weights_path),
    share=False,
    example_image=str(example_path),
    preferred_device=PREFERRED_DEVICE,
    predict_batch_size=BATCH_SIZE,
    use_half=USE_HALF,
)
